In [1]:
import pandas as pd
from binance.client import Client
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
from tqdm import tqdm
import requests
import json
from datetime import datetime, date
from pybit.unified_trading import HTTP

In [2]:
BASKET = [
    "ETH",
    "BNB",
    "SOL",
    "AVAX",
    "MATIC",
    "LUNA",
    "TRX",
    "DOGE",
    "XRP",
    "LINK",
    "ADA",
    "LTC",
    "BTC",
]

In [6]:
BASKET.sort()

<hr>

## Market data

In [3]:
startDT = datetime(2021, 1, 1, 0, 0, 0)
endDT = datetime(2024, 3, 1, 0, 0, 0)
date_range = pd.date_range(startDT, endDT, freq="1d")
session = HTTP()

url = "https://api-testnet.bybit.com/v5/market/kline"

market_data = {}

col_dict = {
    0: "startTime",
    1: "openPrice",
    2: "highPrice",
    3: "lowPrice",
    4: "closePrice",
    5: "volume",
    6: "turnover",
}


for coin in BASKET:
    res_df = pd.DataFrame()
    for d in tqdm(date_range):

        try:
            startTime = int(d.timestamp()) * 1000
            endTime = int((d + pd.offsets.DateOffset(1)).timestamp()) * 1000

            data = session.get_kline(
                category="inverse", symbol=f"{coin}USDT", interval=15, start=startTime, end=endTime
            ).get("result")
            
            df = pd.DataFrame(data)["list"].apply(pd.Series)
            df = df.rename(col_dict, axis=1)
            df["datetime"] = pd.to_datetime(df["startTime"], unit="ms")
            df = df.sort_values("datetime")

            res_df = pd.concat([res_df, df])

        except:
            pass

    res_df.to_csv(f"Bybit_{coin}_market_data.csv")

100%|███████████████████████████████████████| 1156/1156 [01:42<00:00, 11.28it/s]


<hr>

### Funding Rate

In [4]:
startDT=datetime(2021, 1, 1, 0, 0, 0)
endDT=datetime(2024, 3, 1, 0, 0, 0)
date_range = pd.date_range(startDT, endDT, freq="5d")

url = "https://api-testnet.bybit.com/v5/market/funding/history"

for coin in BASKET:
    
    res_df = pd.DataFrame()
    for d in tqdm(date_range):
        
        try:
            startTime = int(d.timestamp()) * 1000
            endTime=int((d + pd.offsets.DateOffset(5)).timestamp()) * 1000

            payload = {
                "category": "inverse",
                "symbol": f"{coin}USDT",
                "limit": 200,
                "startTime": startTime,
                "endTime": endTime,
            }

            response = requests.request(
                "GET",
                url,
                params=payload,
            )

            data = json.loads(response.text)

            df = pd.DataFrame(data["result"]["list"])
            df["datetime"] = pd.to_datetime(df["fundingRateTimestamp"], unit="ms")
            df = df.sort_values('datetime')

            res_df = pd.concat([res_df, df])
        except:
            pass
    
    res_df.to_csv(f"Bybit_{coin}_funding_rate.csv")

100%|█████████████████████████████████████████| 232/232 [00:26<00:00,  8.79it/s]


<hr>

## Open Interest

In [5]:
startDT=datetime(2021, 1, 1, 0, 0, 0)
endDT=datetime(2024, 3, 1, 0, 0, 0)
date_range = pd.date_range(startDT, endDT, freq="1d")

url = "https://api-testnet.bybit.com/v5/market/open-interest"

open_interest = {}

for coin in BASKET:
    
    res_df = pd.DataFrame()
    for d in tqdm(date_range):
        
        try:
            startTime = int(d.timestamp()) * 1000
            endTime=int((d + pd.offsets.DateOffset(1)).timestamp()) * 1000

            payload = {
                "category": "inverse",
                "symbol": f"{coin}USDT",
                "intervalTime": "15min",
                "startTime": startTime,
                "endTime": endTime,
            }

            response = requests.request(
                "GET",
                url,
                params=payload,
            )

            data = json.loads(response.text)

            df = pd.DataFrame(data["result"]["list"])
            df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
            df = df.sort_values('datetime')

            res_df = pd.concat([res_df, df])
        except:
            pass
    
    res_df.to_csv(f"Bybit_{coin}_open_interest.csv")

100%|███████████████████████████████████████| 1156/1156 [02:16<00:00,  8.48it/s]
